### Infêrência Estatística
# Título: Avaliação do impacto da expressões multipalavras nos modelos de linguagem

In [1]:
import markovify
import os
import time
import json
import nltk
import math
import numpy as np

Baseamos o código no Read me da biblioteca "Markovify" em: https://github.com/jsvine/markovify/blob/master/README.md

Para cada arquivos do diretórios especificado lemos e preprocessamos o texto, criamos uma rede de markov com os n-grams (state_size) e combinamos os modelos de todos os arquivos.

In [2]:
# Primeiro, criamos uma função para preprocessar os textos
def preprocess(text):
    # Decodificar usando ISO 8859-1 
    text = text.decode('ISO 8859-1')
    # Colocar todas as letras em minúscula
    text = text.lower()
    # Separar as sentenças pelos pontos de finais
    text = text.split('.')
    new_text = []
    for sent in text:
        # Eliminamos as sentenças com menos de 100 caracteres
        if len(sent)> 100:
            # Excluir os espaços no início e fim das sentenças
            new_text.append(sent.strip())
    return(new_text)

In [3]:
# Contador de tempo
start_time = time.time()

# Cadeia de Markov combinada
combined_model = None

# Contagem de arquivos
n_files = 0

# Iterar pelos diretórios e arquivos
for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        n_files += 1
        
        try:
            # Ler o arquivo e criar uma cadeia de Markov para cada arquivo
            with open(os.path.join(dirpath, filename), 'rb') as f:
                # Preprocessar o texto lido
                text = preprocess(f.read())
                model = markovify.NewlineText(text, retain_original=False, state_size=3)

                # Combinar modelo criado para um arquivo com a Cadeia de Markov combinada
                if combined_model:
                    combined_model = markovify.combine(models=[combined_model, model])
                else:
                    combined_model = model
        except:
            print("Não foi possível ler o arquivo", filename)
            #pass
                    
        # Contador de arquivos processados
        if n_files % 50 == 0:
            print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))

# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50 arquivos processados --- --- 17.889095067977905 seconds ---
--- 100 arquivos processados --- --- 73.62001895904541 seconds ---
--- 139.53809070587158 seconds ---


In [4]:
# Imprimir um exemplo de sentença aleatória
print(combined_model.make_sentence(max_words=50))

 a s   e x p e r i ê n c i a s   m o s t r a m   q u e   e m   2 0 1 5   o   p e t r ó l e o   e   s e u   d e r i v a d o s   t e m   s i d o   i n d u b i t a v e l m e n t e   a s    c o m m o d i t i e s   m a i s   n e r v o s a s   d e s t e   f i n a l   d e   s é c u l o ;   d e s i n t e g r a ç ã o   t e m   e n c o r a j a d o   a   d e s i n t e g r a ç ã o   d e   a t i v i d a d e s ,   f a z e n d o   c o m   q u e   e s s a   n ã o   s e j a   u m a   f a l h a   t ã o   g r a v e   q u a n t o   a   p r i m e i r a 


In [5]:
# Gravar modelo em JSON em disco
model_json = combined_model.to_json()
with open('MarkovChainModel_s3_base.json', 'w') as f:
    json.dump(model_json, f)

In [6]:
# Carregar modelo em JSON do disco
with open('MarkovChainModel_s3_base.json', 'r') as f:
    model_json = markovify.Text.from_json(json.load(f))

Como os textos usados são em português, foi necessário codificar em 'ISO 8859-1', já o modelo JSON está codificado em UTF16. Portanto, ao ler e consultar os termos do modelo é necessário usar a codificação correta.

In [13]:
# Geração de sentença aleatória que iniciam com n-palavras

# Palavras que serãao usadas como semente
palavra_1 = 'a'
palavra_2 = 'petrobras'
palavra_3 = 'é'

# Ajustando a codificação para ser usado no modelo
palavra_1 = ' ' + palavra_1 + ' '
palavra_2 = ' ' + palavra_2 + ' '
palavra_3 = ' ' + palavra_3
palavra_1 = palavra_1.encode('utf16')[3:]
palavra_2 = palavra_2.encode('utf16')[3:]
palavra_3 = palavra_3.encode('utf16')[4:]
seed = palavra_1 + palavra_2 + palavra_3

#Criando sentença aleatória iniciando com as palavras sementes
print(combined_model.make_sentence_with_start(seed.decode()))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 26: invalid continuation byte

Do modelo criado pela biblioteca Markovify, extraímos a cadeia de Markov e gravamos em um dicionário

In [14]:
# Extrair cadeia de Markov do modelo
chain = combined_model.to_dict()['chain']
# Transformando a cadeia em um dicionário 
dic = {}
for state in json.loads(chain):
    dic[(state[0][0],state[0][1], state[0][2])] = state[1]

Criamos uma função para consultar a cadeia de Markov. Dado duas palavras, extraímos as próximas palavras possíveis, a contagem delas no conjunto de treinamento e a soma total. Dessa forma conseguimos calcular  $p(x_3|x_2,x_1)$.

In [15]:
# Função que dado duas palavras, retorna um dicionários com as probabilidades das palavras seguintes
def query_chain(palavra_1, palavra_2, palavra_3):
    # Selecionar o dicionário cuja entrada é o par de palavras "palavra_1" e "palavra_2"
    d = dic[(palavra_1, palavra_2, palavra_3)]
    new_d = {}
    # Transformar a contagem de palavras em probabilidade
    for w in d:
        new_d[w] = d[w]/sum(d.values())
    return (new_d)

O calculamos a perplexidade para cada sentença. Dada as primeiras palavras, obtivemos do modelo a probabilidade da palavra seguinte. Em seguida, avançamos pela sentença recuperando as probablidades.

In [16]:
# Dada uma sentença, a função retorna uma lista com as entropias calculada para cada trio de palavras
def entropy(sent):
    # Quebra a sentença em tokens
    words = nltk.word_tokenize(sent)
    # Variável para guardar a entropia de cada conjunto de palavras
    ln_prob_Xi = []
    # Janela de iteração pela sentença
    for w in range(len(words)-3):
        # Cálculo a entropia
        try:
            ln_prob_Xi.append(- math.log(query_chain(words[w], words[w+1], words[w+2])[words[w+3]]))
        except:
            pass

    return(np.mean(ln_prob_Xi))

In [17]:
# Função que recebe uma string com o caminho e nome da pasta com os arquivos e retorna a perplexidade

def perplexity(dirpath):

    # Início do contador de tempo
    start_time = time.time()
    
    # Variável para armazenar as entropias
    ent = []

    # Contagem de arquivos
    n_files = 0
    # Iterando pelos arquivos da pasta indicada em
    for (dirpath, _, filenames) in os.walk(dirpath):
        
        for filename in filenames:
            n_files += 1
            try:
                # Ler o arquivo e calcular a entropia
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    # Preprocessar o texto lido
                    text = preprocess(f.read())
                    #Calcular a entropia da sentença
                    for sent in text:
                        ent.append(entropy(sent))

            except:
                print("Não foi possível ler o arquivo", filename)
        
                
            # Contador de arquivos processados
            if n_files % 5 == 0:
                print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
    
    # Transformando a lista de entropias em uma np.array e excluir os valores "nan"
    ent = np.array(ent)
    ent = ent[~np.isnan(ent)]
    
    # Fim do contador de tempo
    print ("--- %s seconds ---" % (time.time() - start_time))
    
    return (math.exp(np.mean(ent)))

In [18]:
perplexity("Originais/teste")

C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- 5 arquivos processados --- --- 2.4339380264282227 seconds ---
--- 10 arquivos processados --- --- 5.162765741348267 seconds ---
--- 7.059710264205933 seconds ---


4.5635113240535405